In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve , roc_auc_score
from sklearn.metrics import confusion_matrix
from statistics import mean
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
%matplotlib inline

In [ ]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
pd.set_option('display.MAX_ROWS' , None)
pd.set_option('display.MAX_COLUMNS' , None)

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
df1 = df.copy()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
len(df)

In [ ]:
df[['cat' + str(i) for i in range(19)]].nunique()

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize = (20,25))
for i in range(11):
    plt.subplot(5,4,i+1)
    sns.histplot(df['cont' + str(i)])

In [ ]:
plt.figure(figsize = (25,30))
for i in range(19):
    plt.subplot(5,4,i+1)
    sns.histplot(df['cat' + str(i)])

In [ ]:
cat_columns = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
       'cat16', 'cat17', 'cat18']

In [ ]:
for item in cat_columns:
    for item2 in cat_columns:
        if item != item2:
            if set(df[item]) == set(df[item2]):
                if (len(df[df[item] == df[item2]])/len(df)*100)>80:
                    print(f"For {item} and {item2} they are highly correlated")
                    

In [ ]:
df.head()

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
test.head()

In [ ]:
plt.figure(figsize = (20,25))
for i in range(11):
    plt.subplot(5,4,i+1)
    sns.histplot(test['cont' + str(i)])

In [ ]:
test[['cat' + str(i) for i in range(19)]].nunique()

In [ ]:
plt.figure(figsize = (25,30))
for i in range(19):
    plt.subplot(5,4,i+1)
    sns.histplot(df['cat' + str(i)])

In [ ]:
plt.figure()
fig ,ax1  = plt.subplots(11,2 , figsize =  (40 ,45))
for i in range(11):
    ax1[i , 0].hist(df['cont' + str(i)])
    ax1[i , 1].hist(test['cont' + str(i)])
        

In [ ]:
plt.figure()
fig ,ax1  = plt.subplots(19,2 , figsize =  (200 ,215))
for i in range(19):
    ax1[i , 0].hist(df['cat' + str(i)])
    ax1[i , 1].hist(test['cat' + str(i)])
        

In [ ]:
for i in range(19):
    A = set(df['cat' + str(i)].unique())
    B = set(test['cat' + str(i)].unique())
    if len(A - B) != 0:
        print(f"cat{str(i)} feature with {len(A - B)} values is different on both train and test set")

In [ ]:
A = set(df['cat10'].unique())
B = set(test['cat10'].unique())

In [ ]:
A - B

In [ ]:
df.columns

In [ ]:
columns_2 = ['cat0' , 'cat13' , 'cat14' , 'cat11' , 'cat12']

In [ ]:
Oencoder = OrdinalEncoder()
encoder = LabelEncoder()

In [ ]:
Bencoder = LabelBinarizer()
scaler = StandardScaler()

In [ ]:
df.drop(columns = ['id' , 'cat10'] , inplace = True)

In [ ]:
test.drop(columns = [ 'id' , 'cat10'] , inplace = True)

In [ ]:
X = df.drop(columns = ['target'])
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

In [ ]:
X_train.head()

In [ ]:
encoded_values = []
for i in ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7','cat8', 'cat9', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15','cat16', 'cat17', 'cat18']:
    encoded_values.append(df.groupby(i)['target'].mean())

In [ ]:
encoded_values

In [ ]:
columns = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17',
       'cat18']

In [ ]:
i = 0
for item in columns:
    X_train[item] = X_train[item].map(encoded_values[i])
    X_test[item] = X_test[item].map(encoded_values[i])
    test[item] = test[item].map(encoded_values[i])
    i = i+1

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
test.head()

In [ ]:
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
test[test.columns] = scaler.transform(test[test.columns])

In [ ]:
X_train.head()

In [ ]:
model_xgb = xgb.XGBClassifier()
model_lgb = lgb.LGBMClassifier(max_depth= 7,num_leaves= 200,n_estimators=200,reg_alpha=8)

In [ ]:
cross_val_score(model_lgb,X_train,y_train,cv = 4,scoring = 'accuracy')

In [ ]:
cv = StratifiedKFold(n_splits= 5 , shuffle = True , random_state= 101)

In [ ]:
auc = []
plt.figure(figsize = (12,12))
for train , tes in cv.split(X_train,y_train):
    predictions = model_lgb.fit(X_train.iloc[train] , y_train.iloc[train]).predict_proba(X_train.iloc[tes])
    fpr , tpr , t = roc_curve(y_train.iloc[tes] , predictions[: , 1])
    plt.plot(fpr , tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC')
    auc.append(roc_auc_score(y_train.iloc[tes] , predictions[:,1]))

In [ ]:
print(f'Minimum roc score is {min(auc)}')
print(f'Average of roc score is {mean(auc)}')
print(f'Maximum of roc score is{max(auc)}')

In [ ]:
model_lgb.fit(X_train , y_train)

In [ ]:
predictions = model_lgb.predict_proba(X_test)

In [ ]:
print(roc_auc_score(y_test , predictions[:,1]))

In [ ]:
p = model_lgb.predict_proba(test)

In [ ]:
result = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv' , index_col = 0)

In [ ]:
result['target'] = p[:,1]

In [ ]:
result.head()

In [ ]:
result.to_csv('result.csv')